In [1]:
import PVGeo
import numpy as np

print('NumPy Version: %s' % np.__version__)
print('PVGeo Version: %s' % PVGeo.__version__)

NumPy Version: 1.13.0
PVGeo Version: 1.1.22


/Users/bane/anaconda3/envs/PVGeoTesting/lib/python3.6/site-packages/ipykernel/__main__.py:273: DeprecationWarning: 'U' mode is deprecated


# Welcome to PVGeo

Thanks for checking out this notebook! We hope this provides some insight on how you can get started using PVGeo in your Python (3) routines. Let's get started!

At the top of this notebook, we import `numpy` and `PVGeo` and display the current environment's version of thos packages. Since PVGeo is still in its infancy, feature development is rapid and new versions get deployed very often. With this in mind, be sure to keep the PVGeo in your environment up to date! 

A simple way to update PVGeo from your Jupyter Notebook:

```py
!pip install --upgrade PVGeo
```

Since this notbook is hosted on Microsoft Azure, we have place some data files in the `data/` directory for you to use. The remainder of this notebook will explore a few examples loading, filter, and writing out data using PVGeo.

# Table to Points

Let's go ahead and load a simple file that has XYZ coordinates and a boolean array for fault presence. This point cloud makes some sort of regular grid, but we have forgotten the deatials of the cell spacings and local coordinate rotations.

We will read in this data with pandas and send it to the PVGeo filter `PointsToPolyData` to create a `vtkPolyData` object (essentially a point cloud).

In [2]:
import pandas as pd

In [3]:
points = pd.read_csv('data/fault_points.csv')
points[0:2]

,X,Y,Z,Fault
0,326819.497,4407450.636,1287.5,0
1,326834.340,4407470.753,1287.5,0


In [4]:
# Convert to vtkPolyData (this assumes first three columns are XYZ)
vtkpoints = PVGeo.filters.PointsToPolyData(points)
vtkpoints

(vtkCommonDataModelPython.vtkPolyData)0x10d7e47c8

# Points to Voxelized Volume


Now we that we have the point-attribute data from `data/fault_points.csv` as a VTK data object, we can use a PVGeo filter to create a volumetric data set of this point cloud so that further volumetric operations can be performed wither by PVGeo or VTK filters.

Remember that these points are rotated and we do not know the cell sizes... this is okay! The `VoxelizePoints` filter from PVGeo will handle the recovory of the coordinate rotation and grid our data **without** running an interpolation scheme. The `VoxelizePoints` filter assumes that the points are structure on some rotated XY-plane with regular cell spacings and does the rest on its own! Check out [`VoxelizePoints` code docs](http://docs.pvgeo.org/en/latest/content/PVGeo/filters/voxelize.html#voxelize-points) for more details.

In [5]:
# The full pipeline method
print('Voxelizing... ', end='')
voxelizer = PVGeo.filters.VoxelizePoints()
grid = voxelizer.Apply(vtkpoints)
print('done.')

# Output the results
print('Recovered Angle (deg.): %.3f' % voxelizer.GetRecoveredAngle())
print('Recovered Cell Sizes: (%.2f, %.2f, %.2f)' % voxelizer.GetSpacing())
grid

Voxelizing... done.
Recovered Angle (deg.): 53.550
Recovered Cell Sizes: (25.00, 25.00, 25.00)


(vtkCommonDataModelPython.vtkUnstructuredGrid)0x10d7e4948

In [6]:
# A simpler method to voxelize
# grid = PVGeo.filters.VoxelizePoints().Apply(vtkpoints)
# grid

# Thresholding
Now that we have a volumetric dataset (`vtkUnstrucutred`), we can perform volumetric operations like thresholding, slicing, and the works! We could use VTK's `vtkThreshold` filter but lets try out PVGeo's `PercentThreshold` because it allows us to chop of parts of the data by a given percentage of the data range.

In [7]:
# Threshold: remove the bottom 75%
thresholder = PVGeo.filters.PercentThreshold(percent=75)
thresh = thresholder.Apply(grid, 'Fault')
thresh

(vtkCommonDataModelPython.vtkUnstructuredGrid)0x10d7e4b28

In [8]:
print('Cell reduction: %d vs. %d ' % (grid.GetNumberOfCells(), thresh.GetNumberOfCells()))

Cell reduction: 499200 vs. 89055 


# Slice Volumetric Data

Now lets use one of PVGeo's slicing filters to create slices of the thresholded dataset

In [14]:
slices = PVGeo.filters.ManySlicesAlongAxis(numSlices=5).Apply(grid)
slices

(vtkCommonDataModelPython.vtkMultiBlockDataSet)0x10e2bca08

In [15]:
slice0 = slices.GetBlock(0)
slice0

(vtkCommonDataModelPython.vtkPolyData)0x10e2bc828

In [20]:
# Save slices as png files
import vtk
from vtk.numpy_interface import dataset_adapter as dsa
pdi = slice0

In [23]:
bounds = pdi.GetBounds()
x = (bounds[1] - bounds[0])/2
y = (bounds[3] - bounds[2])/2
z = (bounds[5] - bounds[4])/2
origin = (x, y, z)

def GetSpacing(pdi):
    # First check field data for this info
    
    # Otherwise compute ir
    wpdi = dsa.WrapDataObject(pdi)
    pts = wpdi.Points
    dx, dy, dz = np.diff(pts[:,0]),np.diff(pts[:,1]),np.diff(pts[:,2])
    dx, dy, dz = np.unique(dx),np.unique(dy),np.unique(dz)
    print(dx, dy, dz)
    
#GetSpacing(pdi)
spacing = (25., 25., 25.)

In [28]:
# polygonal data --> image stencil:
pol2stenc = vtk.vtkPolyDataToImageStencil()
pol2stenc.SetInputDataObject(slice0)

bounds = slice0.GetBounds()
pol2stenc.SetOutputOrigin(origin)
pol2stenc.SetOutputSpacing(spacing)
#pol2stenc.SetOutputWholeExtent(pdi.GetWholeExtent())
pol2stenc.Update()
out = pol2stenc.GetOutputDataObject(0)
out

(vtkImagingCorePython.vtkImageStencilData)0x10cbfa408

In [32]:
# cut the corresponding white image and set the background:
imgstenc = vtk.vtkImageStencil()
imgstenc.SetInputDataObject(grid);
imgstenc.SetStencilConnection(pol2stenc.GetOutputPort())
imgstenc.ReverseStencilOff()
imgstenc.Update()

In [ ]:
writer = vtk.vtkMetaImageWriter()
writer.SetFileName("testimage.mhd");
writer.SetInputData(imgstenc.GetOutput());
writer.Write()